Suppose $G$ is a permutation group of $X$, given as a set of generators $Y$, let $\alpha \in X$ and let $T$ be a complete set of left coset representatives of $G_\alpha$ in $G$. Let the surjective map $\phi: G \to T$
be defined via $g\alpha = \phi(g)\alpha$.

Let $x \in G_{\alpha}$. Write $x = y_r \cdots y_1$ with each $y_i \in Y$. Let $t_1 \in T$ be the element belonging to $G_\alpha$. Let $t_{i+1} = \phi(y_it_i)$ for $i = 1, 2, \dots, r$.

We have $t_{i+1}\alpha = \phi(y_it_i)\alpha = (y_it_i)\alpha$. Note that we can telescope to get
\begin{equation}
    t_{r+1}\alpha = (y_rt_r)\alpha = y_r(t_r\alpha) = y_r(y_{r-1} \cdots (y_1t_1)\alpha) = (y_ry_{r-1} \cdots y_1t_1)\alpha.
\end{equation}
Given that $x = y_r \cdots y_1$. Substituting this into our equation gives $t_{r+1}\alpha = (xt_1)\alpha$

Furthermore, $x$ is an element of the stabiliser $G_\alpha$, and $t_1$ is the element of $T$ that belongs to $G_\alpha$. This means that $x \in G_\alpha$ and $t_1 \in G_\alpha$. Therefore, $xt_1 \in G_\alpha$. By definition of the stabiliser $(xt_1)\alpha = \alpha$

Combining the results $t_{r+1}\alpha = (xt_1)\alpha = \alpha$, this shows that $t_{r+1} \in G_\alpha$. By its definition, $t_{r+1} = \phi(y_rt_r)$ is an element of the set of coset representatives $T$. Since $T$ is a complete set of representatives, it contains exactly one element for each coset. Both $t_1$ and $t_{r+1}$ are in $T$ and in $G_\alpha$, so it follows by uniqueness they must be the same element. Hence $t_1 = t_{r+1}$.

---

Let $S = \{\phi(yt)^{-1}yt : y \in Y, t \in T\}$.

$\langle S \rangle \subseteq G_{\alpha}$: Let $s = \phi(yt)^{-1}yt \in S$ be an arbitrary generator. We act on $\alpha$ with s to get
\begin{equation}
    s\alpha = (\phi(yt)^{-1}yt)\alpha = \phi(yt)^{-1}(yt\alpha).
\end{equation}
Using the defining property $(yt)\alpha = \phi(yt)\alpha$,
\begin{equation}
    s\alpha = \phi(yt)^{-1}(\phi(yt)\alpha) = (\phi(yt)^{-1}\phi(yt))\alpha = α.
\end{equation}
This proves that the element $s$ is in the stabiliser $G+\alpha$. Since $s$ is an arbitrary generator, this shows that $\langle S \rangle \leq G_\alpha$.


$G_\alpha \subseteq \langle S \rangle$: Let $x \in G_\alpha$ be arbitrary. Since $G = \langle Y \rangle$, we can write $x = y_r \cdots y_1$ as a product of generators $y_i \in Y$. Define the sequence $t_i$ with $t_1$ as the representative for $G_\alpha$ and $t_{i+1} = \phi(y_it_i)$. For each $i$, define a Schreier generator $s_i = \phi(y_it_i)^{-1}y_it_i = t_{i+1}^{-1}y_it_i \in $. Rearrange this equation to solve for $y_i$ and substituting this expression for each $y_i$ in the product of $x$, we telescope dowwn to
\begin{equation}
    x = t_{r+1}(s_r \cdots s_1)(t_1^{-1}) = t_1(s_r \cdots s_1)(t_1^{-1}).
\end{equation}
The equation shows that $x$ is the conjugate of an element of $\langle S \rangle$ by the element $t_1$. Thus, $t_1^{-1}xt_1$ is in both $\langle S \rangle$ and $G_\alpha$. The set of all such elements is the group $t_1^{-1}G_\alpha t_1$. Because $t_1 \in G_\alpha$, this is just $G_\alpha$ itself. Therefore, we have shown that $G_\alpha \leq \langle S \rangle$.

We can conclude that the stabiliser $G_\alpha$ is generated by the set $S$.

In [13]:
import collections
import copy

class Permutation:
    '''
    A class to represent a permutation of the set {1, 2, ..., n}.
    '''
    def __init__(self, p_list):
        self._p = list(p_list)
        self.size = len(p_list)

    def __len__(self):
        return self.size

    def __repr__(self):
        return f"Perm({self._p})"

    def __eq__(self, other):
        if not isinstance(other, Permutation):
            return NotImplemented
        return self._p == other._p

    def __getitem__(self, i):
        if not 1 <= i <= self.size:
            raise IndexError(f"Index {i} is out of bounds for size {self.size}.")
        return self._p[i - 1]

    def __mul__(self, other):
        if not isinstance(other, Permutation) or self.size != other.size:
            raise ValueError("Permutations must be of the same size.")
        new_p_list = [self[other[i]] for i in range(1, self.size + 1)]
        return Permutation(new_p_list)

    def inverse(self):
        inv_p_list = [0] * self.size
        for i in range(1, self.size + 1):
            j = self[i]
            inv_p_list[j - 1] = i
        return Permutation(inv_p_list)

    def is_identity(self):
        """Checks if the permutation is the identity."""
        for i in range(self.size):
            if self._p[i] != i + 1:
                return False
        return True

In [14]:
def compute_orbit_with_witnesses(generators, alpha):
    n = generators[0].size
    identity = Permutation(list(range(1, n + 1)))
    queue = collections.deque([alpha])
    witnesses = {alpha: identity}
    while queue:
        current_element = queue.popleft()
        current_witness = witnesses[current_element]
        for gen in generators:
            new_element = gen[current_element]
            if new_element not in witnesses:
                witnesses[new_element] = gen * current_witness
                queue.append(new_element)
    return witnesses

def stripping_algorithm_procedure(initial_generators):
    if not initial_generators: return []
    n = initial_generators[0].size
    gen_array = [[None for _ in range(n)] for _ in range(n)]
    for original_gen in initial_generators:
        p = copy.deepcopy(original_gen)
        for i in range(1, n + 1):
            j = p[i]
            if i == j: continue
            existing_gen = gen_array[i - 1][j - 1]
            if existing_gen is None:
                gen_array[i - 1][j - 1] = p
                break
            else:
                p = existing_gen.inverse() * p
    reduced_generators = []
    for row in gen_array:
        for gen in row:
            if gen is not None:
                reduced_generators.append(gen)
    return reduced_generators

In [15]:
def compute_stabilizer_generators(initial_generators, alpha):
    '''
    Computes a generating set for the stabilizer of alpha.
    Args:
        initial_generators: The generating set for group G.
        alpha: The element to stabilize.
    Returns:
        A reduced generating set for the stabilizer G_alpha.
    '''
    if not initial_generators:
        return []

    # The witness_map stores {beta: witness}, where witness(alpha) = beta.
    witness_map = compute_orbit_with_witnesses(initial_generators, alpha)
    # T is the set of coset representatives (the witnesses).
    T = list(witness_map.values())

    # The phi map is implemented by looking up the witness for g(alpha).
    def phi(g):
        beta = g[alpha]
        return witness_map[beta]

    # Use Schreier's Theorem to get an initial (large) generating set.
    schreier_generators = []
    Y = initial_generators

    for y in Y:
        for t in T:
            g = y * t
            phi_g = phi(g)

            # s = phi(g)⁻¹ * g = phi(yt)⁻¹ * y * t
            s = phi_g.inverse() * g
            if not s.is_identity():
                schreier_generators.append(s)

    # Reduce the set of generators with the Stripping Algorithm.
    if not schreier_generators:
        return []
    reduced_stabilizer_gens = stripping_algorithm_procedure(schreier_generators)
    return reduced_stabilizer_gens

print("--- Example: Stabilizer of element 3 in S3 ---")
y1 = Permutation([2, 1, 3])  # (1 2)
y2 = Permutation([2, 3, 1])  # (1 2 3)
s3_generators = [y1, y2]
stabiliser_gens = compute_stabilizer_generators(s3_generators, 3)

print(f"Resulting reduced generating set for the stabilizer G_3: {stabiliser_gens}")

--- Example: Stabilizer of element 3 in S3 ---
Resulting reduced generating set for the stabilizer G_3: [Perm([2, 1, 3])]


Let $k = |Y|$ be the number of initial generators for the group $G$, let $n = |X|$ be the degree of the permutations and let $m = |\mathop{Orb}(\alpha)|$ be the size of the orbit of $\alpha$. Note that $m \leq n$.

*   Orbit computation: The BFS to find the orbit and witnesses explores $m$ elements. For each element, it iterates through $k$ generators. The dominant operation is permutation multiplication, which is $O(n)$. Therefore, the complexity of this step is $O(kmn)$.

*   Schreier generators: This step involves a nested loop over the $k$ generators of $Y$ and the $m$ coset representatives of $T$. Inside the loop, we perform a constant number of permutation multiplications and inversions, each costing $O(n)$. This results in a complexity of $O(kmn)$. This produces $km$ generators.

*   Stripping algorithm: We know that the complexity of the stripping algorithm is $O(kmn^2)$.

The total complexity is the sum of the complexities of the three stages. The dominant stage is the final stripping algorithm. Therefore, the complexity of the entire algorithm is $O(kmn^2)$.